[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Image Processing with Python

## Local Thresholding

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 03/10/2023 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/ImageProcessingPython/0002SciKitImageBasics.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

from numba import jit, njit

# Image Processing
import skimage as ski

# Machine Learning


# Miscellaneous
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Optional, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatRangeSlider, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [2]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [3]:
# Constants



In [4]:
# Fixel Algorithms Packages


In [5]:
# General Auxiliary Functions



## Image Thresholding

Image Thresholding use thresholding for the task of classification of each pixel.  
Given a set of thresholds $\mathcal{T} = \left\{ {T}_{1}, {T}_{2}, \ldots, \right\}$ it partitions the image into $L = \left| \mathcal{T} \right| + 1$ labels.

The case of a single threshold is called _Binarization_.

There are many approaches for calculating the threshold.  
This notebook focuses on the binary case with Local threshold.

* <font color='brown'>(**#**)</font> Binarization is one of the most common pre processes for _Optical Character Recognition_ (OCR).
* <font color='brown'>(**#**)</font> Commonly the threshold is the result of optimizing the parameter of an objective.
* <font color='brown'>(**#**)</font> The [Document Image Binarization](https://dib.cin.ufpe.br) and [DIBCO](https://users.iit.demokritos.gr/~bgat/DIBCO2009/benchmark) are common data sets for the task.

In [6]:
# Parameters

# imgUrl = 'https://i.imgur.com/b8bBmtT.png' #<! From DIBCO data set
imgUrl = 'https://i.postimg.cc/8PQCVSQK/148749752-88e0661f-4356-45f5-b1b1-bc34cd872164.png' #<! From DIBCO data set

numBins = 200

## Generate Data

This section loads the image used to evaluate the results.  

In [7]:
# Load / Generate Data

mI = ski.io.imread(imgUrl)
mI = ski.util.img_as_float64(mI)
mI = np.mean(mI, axis = 2)


In [ ]:
# Display the Data

hF, hA = plt.subplots(figsize = (8, 6))
hA.imshow(mI, cmap = 'gray')
hA.set_title('Input Image');

## Local Image Thresholding

The approach of _Local Image Thresholding_ is applying a **different** threshold per pixel according to its neighborhood.    

Commonly the _Threshold_ is calculated based on the statistics of the given set of pixels in the neighborhood.   
The actual _thresholding_ is applied to the reference pixel of the neighborhood.

* <font color='brown'>(**#**)</font> Any global method can be used locally on a given window of the image.
* <font color='brown'>(**#**)</font> The local methods are commonly more complex.
* <font color='brown'>(**#**)</font> Working on local pre defined windows means the method still can not handle local features / objects perfectly.


### Local Adaptive Method

This method uses a local statistics (Weighted Mean, Median, etc...) and a constant parameter to apply a local thresholding.

In [9]:
# Local Adaptive Thresholding

#===========================Fill This===========================#
# 1. Implement the Local Adaptive Thresholding Method as a function.
# 2. The method calculates the local weighted mean using a Gaussian Window.  
#    The actual threshold is the weighted mean + shift value.
# 3. The input is an image and the radius of the local neighborhood.
# !! Numba can be used.

def LocalAdaptiveThreshold( mI: np.ndarray, localRadius: int, *, valShift: float = 0.0, σ: Optional[float] = None ) -> float:    
    """
    Calculates the local threshold based on local adaptive thresholding.  
    The threshold is based on the weighted mean (Gaussian) of the neighborhood.  
    The threshold is given by `T(ii, jj) = weightedMean(ii, jj) + valShift`.
    
    Parameters:
    -----------
    mI : np.ndarray
        A gray scale image in the range [0, 1].
    localRadius : int
        Defines the radius of the local neighborhood for the Gaussian Kernel.
    valShift : float (Optional)
        Defines the shift of the mean value to be used as the local threshold.
    σ : float (Optional)
        Defines the standard deviation of the Gaussian Kernel used for weighted mean.  
        By default is set such that `np.ceil(3 * σ) == localRadius`.
        
    Returns:
    --------
    mT : np.ndarrary
        The threshold map.

    Remarks:
    --------
     - Uses `scipy.ndimage.gaussian_filter()` with `mode = 'nearest'` and explicit `radius`.
    """

    # Set the default value for `σ` in case it is `None`
    if σ is None:
        σ = np.floor(localRadius / 3.0)
    
    mT  = sp.ndimage.gaussian_filter(mI, sigma = σ, mode = 'nearest', radius = localRadius) #<! Calculate the local weighted mean
    mT += valShift #<! Calculate the local threshold

    return mT

#===============================================================#

### Analysis of Different Methods

The SciKit Image package offers few local methods:

 - [`skimage.filters.threshold_local()`](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_local) (A generalization of the above).
 - [`skimage.filters.threshold_niblack()`](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_niblack).
 - [`skimage.filters.threshold_sauvola()`](https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_sauvola).

This section runs them all and display the results.

In [41]:
# Settings

# List of Tuples for each method.
# The Tuple is the method name, the function and a dictionary of settings.
lThrOp = [('Input Image', lambda mI, dParams: mI, {}),
          ('Local (Ours)', lambda mI, dParams: mI <= LocalAdaptiveThreshold(mI, **dParams), {'localRadius': 4, 'valShift': -0.1}),
          ('Local (SciKit Learn)', lambda mI, dParams: mI <= ski.filters.threshold_local(mI, **dParams), {'block_size': 5, 'method': 'median', 'offset': -0.05}),
          ('Niblack', lambda mI, dParams: mI >= ski.filters.threshold_niblack(mI, **dParams), {'window_size': 11, 'k': 0.75}),
          ('Sauvola', lambda mI, dParams: mI >= ski.filters.threshold_sauvola(mI, **dParams), {'window_size': 5, 'k': 0.15, 'r': 0.5}),
          ]

In [ ]:
# Display Results

hF, vHA = plt.subplots(nrows = 2, ncols = 3, figsize = (12, 8))
vHA = vHA.flat

for ii, (tuThrOp, hA) in enumerate(zip(lThrOp, vHA)):
    mT = tuThrOp[1](mI, tuThrOp[2])
    hA.imshow(mT, cmap = 'gray')
    hA.set_title(tuThrOp[0])

* <font color='green'>(**@**)</font> Optimize the method parameters for this specific image.